# Prophetでコロナ陽性者予測をしよう

## ライブラリの導入とインポート・初期設定


In [ ]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [ ]:
# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# 表示オプション調整
# numpyの浮動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)

# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format

# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)

# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14

# 乱数の種
random_seed = 123

## データの取得

### 日別陽性者数データ

In [ ]:
# 厚生労働省の公開データ
url = 'https://covid19.mhlw.go.jp/public/opendata/newly_confirmed_cases_daily.csv'

# データ読み込み
df = pd.read_csv(url, parse_dates=[0])

# 結果確認
display(df.head())

## データ加工

In [ ]:
# 東京都で絞り込み
df2 = df.query('Prefecture == "Tokyo"')

# 余分な列の削除
df3 = df2.drop('Prefecture', axis=1)

# 列名変更
df3.columns = ['ds', 'y']

# 結果確認
display(df3.tail())

## 時系列グラフ表示

In [ ]:
# 時系列グラフの描画 
import matplotlib.dates as mdates
fig, ax = plt.subplots(figsize=(10, 5))

# グラフ描画
ax.plot(df3['ds'], df3['y'])

# 日付目盛間隔
# 木曜日ごとに目盛りを表示
#weeks = mdates.WeekdayLocator(byweekday=mdates.TH)
# 月ごとに目盛りを表示
months = mdates.MonthLocator()
ax.xaxis.set_major_locator(months)

# 日付表記を90度回転
ax.tick_params(axis='x', rotation=90)

# 方眼表示など
ax.grid()
ax.set_title('東京都コロナ陽性者数')

# 画面出力
plt.show()

## モデル構築、学習から予測まで

下の条件で、モデルを作ってみます。

* 最終日の14日前を基準日に、訓練データと検証データを分割する
* 二週間先（つまり昨日）までの予測をする
* 結果をグラフ表示する


### 訓練データ・検証データの分割

In [ ]:
import pandas.tseries.offsets as offsets

# 分割日 mdayの設定 (最終日から14日前)
mday = df3['ds'].iloc[-1] - offsets.Day(14)

# 訓練用indexと検証用indexを作る
train_index = df3['ds'] <= mday
test_index = df3['ds'] > mday

# 入力データの分割
x_train = df3[train_index]
x_test = df3[test_index]

# 日付データの分割(グラフ表示用)
dates_test = df3['ds'][test_index]

### モデルの生成・学習

In [ ]:
# ライブラリのimport
from fbprophet import Prophet

# モデルインスタンス生成
m1 = Prophet(yearly_seasonality=False, weekly_seasonality=True, 
    daily_seasonality=False,
    seasonality_mode='multiplicative')

# 学習
m1.fit(x_train)

### 予測と結果のグラフ化

In [ ]:
# 予測用データの作成
future1 = m1.make_future_dataframe(periods=14, freq='D')

# 予測
# 結果はデータフレームで戻ってくる
fcst1 = m1.predict(future1)

In [ ]:
# 訓練データ・検証データ全体のグラフ化
fig, ax = plt.subplots(figsize=(10,6))

# 予測結果のグラフ表示(prophetの関数)
m1.plot(fcst1, ax=ax)
months = mdates.MonthLocator()
ax.xaxis.set_major_locator(months)

# 日付表記を90度回転
ax.tick_params(axis='x', rotation=90)

# タイトル設定など
ax.set_title('陽性者数予測')
ax.set_xlabel('日付')
ax.set_ylabel('陽性者数')

# グラフ表示
plt.show()

予測の精度を議論する以前に、直近の予測値が感染爆発を起こしている実測値を表現できていないことがわかります。このままでは、正しく予測することは難しそうです。

## チューニング

In [ ]:
df4 = df3.query('ds >"2021-06-14"')

In [ ]:
display(df4.head())

In [ ]:
# 分割日 mdayの設定 (最終日から14日前)
mday = df3['ds'].iloc[-1] - offsets.Day(14)

# 訓練用indexと検証用indexを作る
train_index = df4['ds'] <= mday
test_index = df4['ds'] > mday

# 入力データの分割
x_train = df4[train_index]
x_test = df4[test_index]

# 日付データの分割(グラフ表示用)
dates_test = df4['ds'][test_index]

In [ ]:
# モデルインスタンス生成
m2 = Prophet(yearly_seasonality=False, weekly_seasonality=True, 
    daily_seasonality=False,
    seasonality_mode='multiplicative')

# 学習
m2.fit(x_train)

# 予測用データの作成
future2 = m2.make_future_dataframe(periods=14, freq='D')

In [ ]:
# 予測
# 結果はデータフレームで戻ってくる
fcst2 = m2.predict(future2)

# 訓練データ・検証データ全体のグラフ化
fig, ax = plt.subplots(figsize=(10,6))

# 予測結果のグラフ表示(prophetの関数)
m2.plot(fcst2, ax=ax)
weeks = mdates.WeekdayLocator(byweekday=mdates.TH)
ax.xaxis.set_major_locator(weeks)

# 日付表記を90度回転
ax.tick_params(axis='x', rotation=90)

# タイトル設定など
ax.set_title('陽性者数予測')
ax.set_xlabel('日付')
ax.set_ylabel('陽性者数')

# グラフ表示
plt.show()

In [ ]:
# ypred2: fcst1から予測部分のみ抽出する
ypred2 = fcst2[-14:][['yhat']].values

# ytest1: 予測期間中の正解データ
ytest2 = x_test['y'].values

In [ ]:
# 時系列グラフの描画 
import matplotlib.dates as mdates
fig, ax = plt.subplots(figsize=(8, 4))

# グラフ描画
ax.plot(dates_test, ytest2, label='正解データ', c='k')
ax.plot(dates_test, ypred2, label='予測結果', c='b')

# 日付目盛間隔
# 木曜日ごとに日付を表示
weeks = mdates.WeekdayLocator(byweekday=mdates.TH)
ax.xaxis.set_major_locator(weeks)

# 日付表記を90度回転
ax.tick_params(axis='x', rotation=90)

# 方眼表示など
ax.grid()
ax.legend()
ax.set_title('コロナ陽性者数予測')

# 画面出力
plt.show()

## おまけ

In [ ]:
# 祝日データ取得
url2 = 'https://raw.githubusercontent.com/holiday-jp/holiday_jp/master/holidays.yml'
df5 = pd.read_csv(url2, sep=':\s+', parse_dates=[0],names=['日付','祝日名'])
df5 = df2[1:]

In [ ]:
display(df5.head())
display(df5.tail())

In [ ]:
# 2020-2021に絞り込み
df2 = df2.query('日付>"2019-12-31" & 日付<"2021-12-31"')

In [ ]:
display(df5.head())
display(df5.tail())